In [ ]:
import glob
import opencc
import copy
from IPython.display import display as print
from typing import List, Dict, Tuple, Optional,Union
from fontTools.ttLib import TTFont
from pathlib import Path
import cv2
from PIL import ImageFont, Image, ImageDraw
import textwrap
import glob
from tqdm.notebook import tqdm
import os
from neko_sdk.ocr_modules.charset.etc_cset import latin62 
import importlib
importlib.reload(mth_tool)

from util.mth_tool import make_alphabet, make_alphabet_v2
from util.mth_tool import get_char_img_from_label, get_char_set
from util.mth_tool import find_char_in_fonts
from util.mth_tool import make_dict, cal_bounding_box
from util.mth_tool import quick_lmdb_for_mth, split_res_dict_for_matplot
import os
import regex
from osocr_tasks.tasksg1.dscs import makept_for_MTH

from neko_sdk.lmdb_wrappers.im_lmdb_wrapper import ImLmdbWrapper
import glob
import os.path
import cv2
from pathlib import Path
from util import mth_tool

db_names = ["MTH1000", "MTH1200", "TKH"]


In [ ]:

for db_name in db_names:
    # _rr = make_alphabet_v2(glob.glob(f"ddata/TKHMTH2200/{db_name}/label_textline/*.txt"),f"runtime/alphabet_{db_name}_ssss.txt")
    pass

In [ ]:
import random
import pickle

train_ratio = 0.75
font_paths = glob.glob("ddata/fonts/HanaMin/**/*.ttf", recursive=True)
res_apl = {}
res_apl_path = "runtime/res_apl.pt"

for db_name in db_names:
    db_info = {}

    text_file_list = glob.glob(f"ddata/TKHMTH2200/{db_name}/label_textline/*.txt")
    random.shuffle(text_file_list)

    train_n = int(len(text_file_list) * train_ratio)
    train_list = text_file_list[:train_n]
    test_all_list = text_file_list[train_n:]
    db_info['train_list'] = train_list
    db_info['test_all_list'] = test_all_list

    train_alphabet = f"ddata/recdatassd/{db_name}/alphabet_{db_name}_train.txt"
    _rr = make_alphabet_v2(train_list, train_alphabet)
    db_info['train_alpha'] = _rr

    test_all_alphabet = f"ddata/recdatassd/{db_name}/alphabet_{db_name}_test_all.txt"
    _rr = make_alphabet_v2(test_all_list, test_all_alphabet)
    db_info['test_all_alpha'] = _rr

    # make_dict(train_alphabet, font_paths, f"ddata/recdatassd/{db_name}/dict_train.pt", force_rebuild=True)
    res_apl[db_name] = db_info

import torch
torch.save(res_apl, res_apl_path)

# pickle.dump(res_apl, open(res_apl_path, 'w+'))


In [ ]:
if os.path.exists(res_apl_path):
    res_apl = torch.load(res_apl_path)

In [ ]:
# 创建lmdb
for db_name, db_info in res_apl.items():
    gt_file_path_list = db_info['train_list']
    dst = f"ddata/recdatassd/{db_name}_train"
    quick_lmdb_for_mth(gt_file_path_list, dst,lang="None")
    
    gt_file_path_list = db_info['test_all_list']
    dst = f"ddata/recdatassd/{db_name}_test_all"
    quick_lmdb_for_mth(gt_file_path_list, dst,lang="None")
    


In [ ]:
for db_name, db_info in res_apl.items():
    _alphabet = f"ddata/recdatassd/{db_name}_train/alphabet_{db_name}_train.txt"
    make_dict(_alphabet, font_paths, f"ddata/recdatassd/{db_name}_train/dict_train.pt", force_rebuild=False)
    
    _alphabet = f"ddata/recdatassd/{db_name}_test_all/alphabet_{db_name}_test_all.txt"
    make_dict(_alphabet, font_paths, f"ddata/recdatassd/{db_name}_test_all/dict_test_all.pt", force_rebuild=False)


In [ ]:
from matplotlib import plot as plt

fig, axes = plt.subplots(3, 2)
axes = axes.reshape(-1)
i = 0
for db_name, db_info in res_apl.items():
    keyss = ["train_alpha", "test_all_alpha"]
    for key in keyss:
        _alpha = db_info[key]
        X, Y = split_res_dict_for_matplot(_alpha)
        axes[i].bar(range(len(Y)), sorted(Y))
        i+=1
# plt.bar(range(len(Y)), sorted(Y))

In [ ]:
res_dict = get_char_img_from_label()
# aa  = 0
# for char,char_img in res_dict.items():
#     print(char)
#     char_img = cv2.resize(char_img, (64, 64), interpolation=cv2.INTER_LINEAR)
#     print(Image.fromarray(char_img))
#     aa += 1
#     if aa>5:
#         break

In [ ]:

# txt
alphabets = list(res_dict.keys())
alphabets = sorted(alphabets)

text = "".join(alphabets)

# fonts
font_path_list = glob.glob("ddata/fonts/HanaMin/**/*.ttf", recursive=True)

fontSize = 64
ttfonts,imFonts = [],[]
for f_p in font_path_list:
    ttfonts.append( TTFont(f_p))
    imFonts.append(ImageFont.truetype(f_p, size=fontSize))

# font_family = ImageFont.truetype_family(*imFonts)

char_font_ids = []
for char in alphabets:
    ii, _ = find_char_in_fonts(char, ttfonts=ttfonts)
    char_font_ids.append(ii)

fontSize = 64
max_len = 32
lines = textwrap.wrap(text, width=max_len)

FOREGROUND = "#000000"
bground = (255,255,255)

im_font = imFonts[0]
tw, th = im_font.getsize(char)
im = Image.new("RGB", ((fontSize * max_len), 2 * len(lines) * (fontSize + 8)), (255, 255, 255))
dr = ImageDraw.Draw(im)

miss_char = []
x,y=0,0
for i,char,f_id in zip(range(len(alphabets)), alphabets, char_font_ids):
    if f_id is None:
        col = i % max_len
        row = int(i/max_len)
        miss_char.append(f"{row+1},{col}, {char}")
        f_id = 0
    im_font = imFonts[f_id]
    char_img = res_dict[char]
    
    x = tw * (i % max_len)
    y = 2 * th * int(i/max_len)
    
    dr.text((x, y), char, font=im_font, fill=FOREGROUND, spacing=0, align="left")
    char_img = cv2.resize(char_img, (64, 64), interpolation=cv2.INTER_LINEAR)
    char_img = Image.fromarray(char_img)
    im.paste(char_img,(x,y+th))

print(miss_char)
# print(im)
im.save("runtime/alphabets-char-3.jpg")


In [ ]:
font_paths = glob.glob("ddata/fonts/HanaMin/**/*.ttf", recursive=True)

# --------------------------------------------------------------
# db_names = ["MTH1000","MTH1200", "TKH"]
# for db_name in db_names:
#     aplphabets_file_path = f"runtime/alphabet_{db_name}.txt"
#     dst_path = f"ddata/MTH_DB/{db_name}/dict.pt"
#     make_dict(aplphabets_file_path, font_paths, dst_path, force_rebuild=True)

# --------------------------------------------------------------
db_names = ["MTH1000", "MTH1200", "TKH"]
for db_name in db_names:
    make_dict(f"ddata/recdatassd/{db_name}/alphabet_{db_name}.txt", font_paths, f"ddata/recdatassd/{db_name}/dict.pt", force_rebuild=True)


In [ ]:
from PIL import ImageFont, Image, ImageDraw
import textwrap
import glob
from util.mth_tool import render_char

# render_char()

In [ ]:
# mth_dataset_dir = "/home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/MTH1200"
# dst = "/home/izhangxm/Desktop/work/main/VSDF/ddata/lmdb/MTH1200"
# quick_lmdb(mth_dataset_dir, dst)

# mth_dataset_dir = "/home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/MTH1000"
# dst = "/home/izhangxm/Desktop/work/main/VSDF/ddata/lmdb/MTH1000"
# quick_lmdb(mth_dataset_dir, dst)

# mth_dataset_dir = "/home/izhangxm/Desktop/work/main/VSDF/ddata/TKHMTH2200/TKH"
# dst = "/home/izhangxm/Desktop/work/main/VSDF/ddata/lmdb/TKH"
# quick_lmdb(mth_dataset_dir, dst)